In [3]:
import pandas as pd
import numpy as np
import smtplib
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

# 1. Simulate Weather Data (Date and Temperature)
# You can replace this with real weather data from APIs like OpenWeatherMap
data = {
    'Date': pd.date_range(start='2023-06-01', periods=100, freq='D'),
    'Temperature': np.random.randint(28, 45, size=100)  # Simulated temperatures
}
df = pd.DataFrame(data)

# Define a heat wave as 3+ consecutive days where temp > 35°C
def label_heat_wave(temps, threshold=35, consecutive_days=3):
    """Label the data with heat wave flag based on consecutive days exceeding threshold."""
    heat_wave = np.zeros_like(temps)
    count = 0
    for i in range(len(temps)):
        if temps[i] > threshold:
            count += 1
            if count >= consecutive_days:
                heat_wave[i] = 1
        else:
            count = 0
    return heat_wave

df['HeatWave'] = label_heat_wave(df['Temperature'])

# 2. Prepare data for training (predict heat wave based on temperature)
df['TempLag1'] = df['Temperature'].shift(1)
df['TempLag2'] = df['Temperature'].shift(2)
df = df.dropna()  # Drop rows with NaN (due to lag features)

# Features and labels
X = df[['Temperature', 'TempLag1', 'TempLag2']]
y = df['HeatWave']

# Split data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 3. Train a predictive model (Random Forest)
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Evaluate the model
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Model Accuracy: {accuracy * 100:.2f}%")

# 4. Send Mobile Alerts for Heat Waves (using email as an example for simplicity)
# You can use services like Twilio or Firebase for real mobile notifications.

def send_alert(email_address, subject, message):
    """Send an email alert (use Twilio or other services for real mobile alerts)."""
    # Email settings (use your email and SMTP server details)
    smtp_server = "smtp.gmail.com"
    smtp_port = 587
    sender_email = "your_email@gmail.com"
    password = "your_password"

    try:
        # Set up the server
        server = smtplib.SMTP(smtp_server, smtp_port)
        server.starttls()
        server.login(sender_email, password)

        # Create the email
        email_message = f"Subject: {subject}\n\n{message}"

        # Send the email
        server.sendmail(sender_email, email_address, email_message)
        server.quit()
        print(f"Alert sent to {email_address}")
    except Exception as e:
        print(f"Error sending alert: {e}")

# 5. Simulate prediction and send alerts
def check_for_heat_wave_and_alert(temperature_today, temperature_yesterday, temperature_two_days_ago):
    """Check if there is a heat wave based on the temperatures and send alerts if true."""
    # Use the trained model to predict
    prediction = model.predict([[temperature_today, temperature_yesterday, temperature_two_days_ago]])

    if prediction == 1:
        print("Heat Wave detected!")
        # Example alert (use actual mobile notification service here)
        send_alert('user_email@example.com', 'Heat Wave Alert!', 'A heat wave is expected in your area. Stay safe!')
    else:
        print("No heat wave detected.")

# Test the alert system with sample temperatures
check_for_heat_wave_and_alert(40, 38, 36)  # Change these values to test different scenarios


Model Accuracy: 100.00%
Heat Wave detected!


C:\Users\bhavy\jupyter\Lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(


Error sending alert: (535, b'5.7.8 Username and Password not accepted. For more information, go to\n5.7.8  https://support.google.com/mail/?p=BadCredentials d2e1a72fcca58-71afc939068sm969079b3a.133 - gsmtp')
